# Example for:
Transfer Learning Empirical Experiment from first 5 classes of MNIST into final 5 classes of MNIST
- Almost all the details about optimizers, training, lr, ... are in the params dict!
- And all experiment design details too! reinit, freeze, ..

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64, # num channels for CNN
      'two_linear_layers': False,
      'hidden_dim_lin': 128,  # if two_linear_layers == True
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': [0, 1, 2, 3, 4],
      'fine_tune_classes': [5, 6, 7, 8, 9],
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      # default value for pooling_every_n_layers is 1 (after each cnn layer)
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True,         # CHANGE: reinit the conv lyers only after the cut
      'reinit_both_dense': True   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.MNIST, datasets.MNIST, root_dir, transform=transform)

In [5]:
dataloader_wrapped.finetune_train_loader.dataset.__len__()

26464

## Pretraining

In [6]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=5, bias=True)
)

In [11]:
# Train and evaluate
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 22.05%
Validation Accuracy: 21.90%
Epoch: 1 	Training Accuracy: 79.30%
Validation Accuracy: 79.44%
Epoch: 2 	Training Accuracy: 79.70%
Validation Accuracy: 79.57%
Epoch: 3 	Training Accuracy: 87.36%
Validation Accuracy: 86.40%
Epoch: 4 	Training Accuracy: 90.55%
Validation Accuracy: 90.06%
Epoch: 5 	Training Accuracy: 92.87%
Validation Accuracy: 92.42%
Epoch: 6 	Training Accuracy: 94.19%
Validation Accuracy: 93.79%
Epoch: 7 	Training Accuracy: 95.14%
Validation Accuracy: 94.87%
Epoch: 8 	Training Accuracy: 96.23%
Validation Accuracy: 95.59%
Epoch: 9 	Training Accuracy: 96.83%
Validation Accuracy: 96.08%
Epoch: 10 	Training Accuracy: 97.04%
Validation Accuracy: 96.31%
Epoch: 11 	Training Accuracy: 97.52%
Validation Accuracy: 96.73%
Epoch: 12 	Training Accuracy: 97.84%
Validation Accuracy: 97.19%
Epoch: 13 	Training Accuracy: 98.08%
Validation Accuracy: 97.38%
Epoch: 14 	Training Accuracy: 98.13%
Validation Accuracy: 97.55%
Epoch: 15 	Training Accuracy: 98.40

In [12]:
torch.save(pretrained_model.state_dict(), 'pretrained_models/pretrained_mnist_final/pretrained_model_new.pth')

In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_mnist_final/pretrained_model_new.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=5, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.0260, Accuracy: 3032.0/3059 (99%)

              precision    recall  f1-score   support

     Class 0       1.00      0.99      1.00       544
     Class 1       0.99      0.99      0.99       670
     Class 2       0.98      0.99      0.99       631
     Class 3       1.00      0.98      0.99       613
     Class 4       0.99      1.00      0.99       601

    accuracy                           0.99      3059
   macro avg       0.99      0.99      0.99      3059
weighted avg       0.99      0.99      0.99      3059



0.9911735861392612

In [13]:
# save model for later use
foldername = "pretrained_models/pretrained_mnist_final"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp.get("activation_function", nn.ReLU))
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [9]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 1.0000, Test Accuracy: 0.7176

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 1
Training Accuracy: 1.0000, Test Accuracy: 0.7053

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 2
Early stopping invoked.
Training Accuracy: 0.9200, Test Accuracy: 0.7622

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 3
Training Accuracy: 1.0000, Test Accuracy: 0.7130

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 4
Training Accuracy: 1.0000, Test Accuracy: 0.7089

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 5
Early stopping invoked.
Training Accuracy: 1.0000, Test Accuracy: 0.6921

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 6
Early stopping invoked.
Training Accuracy: 0.5600, Test Accuracy: 0.3433

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 7
Early stopping invoked.
Training Accuracy: 0.7600, Test Accuracy: 0.5175

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 8
Early stopping invoked.
Training Accuracy: 0.92

In [11]:
print(baselines_results)

[{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.7176234525914842}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.7053462113998096}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.92, 'test_acc': 0.7621752142565638}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.7129642225547544}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.7089171541286899}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.6921167188137668}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.56, 'test_acc': 0.3433206366480751}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repe

In [21]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
baselines_results = [params_tmp] + baselines_results

with open(f'results_jsons/baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}.json', 'w') as f:
    json.dump(baselines_results, f)

### Fine-tuning

In [8]:
results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [12]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9608, Test Accuracy: 0.9531

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9602, Test Accuracy: 0.9539

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 2
Training Accuracy: 0.9598, Test Accuracy: 0.9525

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 3
Training Accuracy: 0.9558, Test Accuracy: 0.9549

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 4
Training Accuracy: 0.9613, Test Accuracy: 0.9568

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 5
Training Accuracy: 0.9582, Test Accuracy: 0.9564

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 6
Training Accuracy: 0.9583, Test Accuracy: 0.9543

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.001, Repeat: 7
Training Accuracy: 0.9629, Test Accuracy: 0.9512

Sampled Percentage: 0.3, Sampled Cut Point: 3, Lr: 0.00

In [13]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results_jsons/results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)
results = results[1:]